In [416]:
import pandas as pd
import numpy as np
import string
import os
os.getcwd()

'/home/jia/Desktop/git/STB_social_media_analytics/experimentation/jiaxin_experiment/Ali'

In [417]:
df_raw=pd.read_csv('./200218_151931_sentiment.csv')

In [418]:
df_raw.describe()

,REVIEW_INDEX
count,683.000000
mean,341.000000
std,197.309402
min,0.000000
25%,170.500000
50%,341.000000
75%,511.500000
max,682.000000


In [419]:
#remove the invalid API outpus
pos = df_raw['OVERALL']=='POS'
neg = df_raw['OVERALL']=='NEG'
neu = df_raw['OVERALL']=='NEU'
df=df_raw[neu].append(df_raw[pos].append(df_raw[neg]))

In [420]:
#make all entries string, to turn into list
a=list(map(lambda x: str(x), df['ASPECTS']))
b=list(map(lambda x: str(x), df['ASPECTS_POLARITIES']))



In [421]:
#obtain li_asp: a list containing lists, each list element is the aspects list obtained from a review
c=[]
for i in range(len(a)):
    c+=[a[i].split('","')]
li_asp=[]
length=[]
for i in range(len(c)):
    li1=[]
    for j in range(len(c[i])):
        li1+=[c[i][j].translate(c[i][j].maketrans('','',string.punctuation))]
    li_asp+=[li1]
    length+=[len(li1)]


In [422]:
#obtain li_pol: a list containing lists, each list element is the polarity list obtained from a review
#exact same length with the li_asp
c=[]
for i in range(len(b)):
    c+=[b[i].split('","')]
li_pol=[]
for i in range(len(c)):
    li1=[]
    for j in range(len(c[i])):
        li1+=[c[i][j].translate(c[i][j].maketrans('','',string.punctuation))]
    li_pol+=[li1]
li_asp

[['里面', '买票', '餐厅'],
 ['室外', '风景', '放松心情', '这里'],
 ['出票', '站在门口现买', '价格', '出票速度', '出票速度'],
 ['植物园', '高科技', '植物', '花香', '树'],
 ['交通', '空气', '野餐', '热带植物', '空气', '里面', '地铁'],
 [''],
 ['胡姬花', '带小孩'],
 [''],
 ['理', '观赏性', '花草', '植物园', '植物园', '非常', '人'],
 ['蝴蝶兰', '花园'],
 ['许多植物', '看'],
 [''],
 ['姬花园', '胡姬花'],
 ['', '色', '风光', '价格', '门票价格', '门票价格', '景色', '景色', '人', '秀'],
 ['植物园', '免费观光', '有一个', '里面'],
 ['植物园', '热带的植物', '比较', '比较'],
 ['纪念', '食物', '门票', '地方', '逛'],
 ['雨雾林', '植物园'],
 ['植物园', '花园里', '胡姬花', '进园'],
 ['景色', '植物', '创意', '根雕', '非常', '凉'],
 [''],
 ['价格', '里边', '看', '话'],
 ['路线设计', '设计', '公园', '路线'],
 [''],
 ['植物'],
 ['价格', '出票'],
 ['小商店纪念品', '质量', '花草'],
 ['看起来', '草地'],
 ['空气', '免费的', '外面', '买票'],
 ['兰花', '携程购票', '购票', '安排', '票'],
 ['景点', 'mm', '景点', '兰花', '花'],
 ['花草', '人'],
 ['买套票', '非常', '非常', '非常'],
 ['景色'],
 ['景色', '一', '静', '里面', '里面', '景色', '人', '车'],
 ['以来', '兰花', '话'],
 ['坐车', '花园城市', '比较', '非常'],
 ['植物园', '天然绿色', '植物', '感觉', '凉'],
 ['植物园', '当地人', '园区', '花'],
 ['植物', '花朵', '交通

In [423]:
#put back to df, delete the rows with empty API output
df['ASPECTS']=li_asp
df['ASPECTS_POLARITIES']=li_pol
col_temp=list(range(36))
col=list(map(lambda x: 'ASPECT'+str(x),col_temp))
df[col] = pd.DataFrame(df.ASPECTS.values.tolist(), index= df.index)
col_pol=list(map(lambda x: 'ASPECT_POLARITY'+str(x),col_temp))
df[col_pol] = pd.DataFrame(df.ASPECTS_POLARITIES.values.tolist(), index= df.index)

ValueError: Columns must be same length as key

In [395]:
df= df[df['ASPECT0'] !='']
df= df[df['ASPECTS_POLARITIES'] != '']
df=df.reset_index()


In [396]:
#map the aspects and polarities, for each list elements, and assign to new column
pair_list=[]
for i in range(len(df['ASPECTS'])):
    row_pair=[]
    for j in range(len(df['ASPECTS'][i])):
        row_pair+=[df['ASPECTS'][i][j]+','+df['ASPECTS_POLARITIES'][i][j]]
    pair_list+=[row_pair]

In [397]:
df['PAIR_LIST']=pair_list
df=df.drop(col+col_pol+['ASPECTS','ASPECTS_POLARITIES'],axis=1)

In [398]:
#separate each pair into a column, prepare for change to long format
pair=list(map(lambda x: 'PAIR'+str(x),col_temp))
df[pair] = pd.DataFrame(df.PAIR_LIST.values.tolist(), index= df.index)
df=df.sort_values(by='REVIEW_INDEX')


In [399]:
long = pd.melt(df,id_vars=id_vars[0:-1],value_vars=pair)
long=long.sort_values(by='REVIEW_INDEX')
long=long.reset_index()

In [400]:
long=long.dropna()
long=long.drop(['level_0','index','variable'],axis=1)
long=long.reset_index(drop=True)

In [403]:
long['value']=pd.Series(long['value'])

In [404]:
long[['ASPECT','POLARITY']] = long.value.str.split(",",expand=True,)

In [406]:
long=long.drop(['value'],axis=1)
long

,REVIEW_INDEX,OVERALL,POLARITY_DIST: CONSUMPTION,POLARITY_DIST: ENVIRONMENT,POLARITY_DIST: F&B,POLARITY_DIST: HYGIENE,POLARITY_DIST: SAFETY,POLARITY_DIST: SERVICE,POLARITY_DIST: SHOPPING,POLARITY_DIST: SIGHTSEEING,POLARITY_DIST: TRANSPORTATION,ASP_NEG_RIO,ASP_NEU_RIO,ASP_POS_RIO,REV_NEG_RIO,REV_NEU_RIO,REV_POS_RIO,TOTAL_ASPECTS,ASPECT,POLARITY
0,0,POS,"[0,0,0]","[2,1,2]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","2,0,13]","[0,0,0]",0.19999900000499998,0.049999750001249996,0.7499962500187499,0.0,0.0,0.9999000099990001,20,滨海湾花园,POS
1,0,POS,"[0,0,0]","[2,1,2]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","2,0,13]","[0,0,0]",0.19999900000499998,0.049999750001249996,0.7499962500187499,0.0,0.0,0.9999000099990001,20,flowerdome,NEG
2,0,POS,"[0,0,0]","[2,1,2]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","2,0,13]","[0,0,0]",0.19999900000499998,0.049999750001249996,0.7499962500187499,0.0,0.0,0.9999000099990001,20,玻璃花园,POS
3,0,POS,"[0,0,0]","[2,1,2]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","2,0,13]","[0,0,0]",0.19999900000499998,0.049999750001249996,0.7499962500187499,0.0,0.0,0.9999000099990001,20,室内冷气,POS
4,0,POS,"[0,0,0]","[2,1,2]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","2,0,13]","[0,0,0]",0.19999900000499998,0.049999750001249996,0.7499962500187499,0.0,0.0,0.9999000099990001,20,非常,NEU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12753,2999,POS,"[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","0,1,4]","[0,0,0]",0.0,0.1999960000799984,0.7999840003199936,0.0,0.0,0.9999000099990001,5,看,NEU
12754,2999,POS,"[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","0,1,4]","[0,0,0]",0.0,0.1999960000799984,0.7999840003199936,0.0,0.0,0.9999000099990001,5,乐,POS
12755,2999,POS,"[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","0,1,4]","[0,0,0]",0.0,0.1999960000799984,0.7999840003199936,0.0,0.0,0.9999000099990001,5,光,POS
12756,2999,POS,"[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","[0,0,0]","0,1,4]","[0,0,0]",0.0,0.1999960000799984,0.7999840003199936,0.0,0.0,0.9999000099990001,5,景色,POS


In [407]:
long.to_csv('./BG_long.csv')